# AgBr Complex

In [46]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt; rootpath=txt(1:n-11);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
%graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")

In [48]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=1; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=2; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
%database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

AgT=1e-6; BrT=1e-3; pH=7; 

TOTALS=[AgT BrT];
TOTALS(TOTALS==0)=1e-16; % get rid of zero values, b/c div by zero error
%TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;
pH=pH; pe=20.75-pH; 

tic;
[Ag,Br,AgBrs,MASSERR]=AgBrtableau(pH,pe,TOTALS',flag1,flag2,flag3,flag4,flag5);
tableautime=toc;

Agmasserror=MASSERR(1); Brmasserror=MASSERR(2);

X no ITER EXEED no solid logX


In [49]:
% MODEL USING PHREEQC

minerals=[{'AgBrs'}]; totalvector=[AgT; BrT; BrT]; totalnames=[{'Ag'}; {'Br'}; {'Na'}]; speciesexport=[{'Ag+'}; {'Br-'};{'AgBr'};{'AgBr2-'};{'AgBr3-2'};{'AgBr4-3'}];
%database=['llnl_nosolubleAgCl.dat']; 
acid=['NaOH']; show=0; T=25;
%database=['AgCl.dat']; 
database=['complexAgBr.dat']; 

tic
[solutionspeciesconcs, speciesnames, solidconcs, solidnames]=runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
PHREEQCtime=toc;

AgPHREEQC=solutionspeciesconcs(1); 
BrPHREEQC=solutionspeciesconcs(2); 
AgBrPHREEQC=solutionspeciesconcs(3); 
AgBr2PHREEQC=solutionspeciesconcs(4); 
AgBr3PHREEQC=solutionspeciesconcs(5); 
AgBr4PHREEQC=solutionspeciesconcs(6);
AgBrsPHREEQC=solidconcs(1)

AgmasserrorPHREEQC=AgT-AgPHREEQC-AgBrsPHREEQC-AgBrPHREEQC-AgBr2PHREEQC-AgBr3PHREEQC-AgBr4PHREEQC; 
BrmasserrorPHREEQC=BrT-BrPHREEQC-AgBrsPHREEQC-AgBrPHREEQC-2*AgBr2PHREEQC-3*AgBr3PHREEQC-3*AgBr4PHREEQC;

Input file: runphreeqc.txt

Output file: out.txt

Database file: complexAgBr.dat


Initializing...                                                                 

End of Run after 0.003468 Seconds.
AgBrsPHREEQC = 9.6362e-07


In [ ]:
% exact model

Ksp=10^-12.30;

% check SI

IAP=AgT*BrT;

Agexact=AgT; Brexact=BrT; AgBrsexact=0;

if IAP>=Ksp
a=1; b=BrT-AgT; c=-Ksp;
t=roots([a b c]); t=t(t>0); t=real(t(imag(t)==0));
Agexact=t; AgBrsexact=AgT-Ag; Brexact=BrT-AgBrsexact;
end


In [52]:
% compare models

Agcompare=[Ag AgPHREEQC Agexact]
Brcompare=[Br BrPHREEQC Brexact]
AgBrcompare=[AgBrs AgBrsPHREEQC AgBrsexact]
Agmasserrorcompare=[Agmasserror AgmasserrorPHREEQC]
Brmasserrorcompare=[Brmasserror BrmasserrorPHREEQC]
timecompare=[tableautime PHREEQCtime]

Agcompare =

   5.0169e-10   5.3896e-10   1.0000e-06

Brcompare =

   9.9900e-04   9.9898e-04   1.0000e-03

AgBrcompare =

   9.9950e-07   9.6362e-07            0

Agmasserrorcompare =

            0  -1.3439e-19

Brmasserrorcompare =

   6.5052e-14   3.0926e-13

timecompare =

   5.8705e-01   2.3809e-02

